# Introducción al Aprendizaje Automático.

## Subajuste y sobreajuste

### Experimentando con diferentes modelos

Al final de esta lección, comprenderemos los conceptos de subajuste y sobreajuste (*underfitting* y *overfitting*) y podremos aplicar estas ideas para que nuestros modelos sean más precisos.

Ahora que disponemos de una forma confiable de medir la precisión del modelo, podemos experimentar con modelos alternativos y ver cuál ofrece las mejores predicciones. Pero, ¿qué alternativas tenemos para los modelos?

Podemos ver en la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) de scikit-learn que el modelo de árbol de decisión tiene muchas opciones (más de lo que querremos o necesitaremos durante mucho tiempo). Las opciones más importantes determinan la profundidad del árbol. Recuerdemos que la profundidad de un árbol es una medida de cuántas divisiones hace antes de llegar a una predicción. Este es un árbol relativamente poco profundo:

![Arbol profundo](./images/arbol_profundo.png)

En la práctica, no es raro que un árbol tenga 10 divisiones entre el nivel superior (todas las casas) y una hoja. A medida que el árbol se hace más profundo, el conjunto de datos se divide en hojas con menos casas. Si un árbol solo tiene 1 división, divide los datos en 2 grupos. Si cada grupo se divide nuevamente, obtendríamos 4 grupos de casas. Dividir cada uno de esos nuevamente crearía 8 grupos. Si seguimos duplicando el número de grupos agregando más divisiones en cada nivel, tendremos 2<sup>10</sup> grupos de casas para cuando lleguemos al décimo nivel. Eso es 1024 hojas.

Cuando dividimos las casas entre muchas hojas, también tenemos menos casas en cada hoja. Las hojas con muy pocas casas harán predicciones muy cercanas a los valores reales de esas casas, pero pueden hacer predicciones muy poco confiables para nuevos datos (porque cada predicción se basa en solo unas pocas casas).

Este es un fenómeno llamado **sobreajuste** (*overfitting*), donde un modelo coincide con los datos de entrenamiento casi a la perfección, pero no funciona bien en la validación y otros datos nuevos. Por otro lado, si hacemos que nuestro árbol sea muy poco profundo, no divide las casas en grupos muy distintos.

En un extremo, si un árbol divide las casas en solo 2 ó 4, cada grupo todavía tiene una gran variedad de casas. Las predicciones resultantes pueden estar muy lejos para la mayoría de las casas, incluso en los datos de entrenamiento (y también será malo en la validación por la misma razón). Cuando un modelo no logra capturar distinciones y patrones importantes en los datos, por lo tanto, se desempeña mal incluso en los datos de entrenamiento, lo que se denomina adaptación insuficiente o **subajuste** (*underfitting*).

Dado que nos preocupamos por la precisión de los nuevos datos, que estimamos a partir de nuestros datos de validación, queremos encontrar el punto óptimo entre el ajuste insuficiente y el sobreajuste. Visualmente, queremos el punto bajo de la curva de validación (roja) en

![Over_Underfitting](./images/over_underfitting.png)

### Ejemplo

Existen algunas alternativas para controlar la profundidad del árbol y muchas permiten que algunas rutas a través del árbol tengan mayor profundidad que otras rutas. Pero el argumento *max_leaf_nodes* proporciona una forma muy sensata de controlar el overfitting vs underfitting. Cuantas más hojas permitimos que haga el modelo, más nos movemos desde el área de underfitting en el gráfico anterior al área de overfitting.

Podemos usar una función de utilidad para ayudar a comparar las puntuaciones de MAE de diferentes valores para *max_leaf_nodes*:

In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
    
# Cargamos los datos
melbourne_file_path = "./input/melbourne-housing-snapshot/melb_data.csv"
melbourne_data = pd.read_csv(melbourne_file_path) 
# Eliminamos las filas con valores de precio ausentes
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Elegimos el objetivo y las características
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

# Divide los datos en datos de entrenamiento y validación, tanto para las características como para el objetivo
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

Podemos usar un bucle for para comparar la precisión de los modelos construidos con diferentes valores de *max_leaf_nodes*.

In [5]:
# Compara el MAE con diferentes valores de max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  347380
Max leaf nodes: 50  		 Mean Absolute Error:  258171
Max leaf nodes: 500  		 Mean Absolute Error:  243495
Max leaf nodes: 5000  		 Mean Absolute Error:  254983


De las opciones enumeradas, 500 es el número óptimo de hojas.

### Conclusión

Las modelos pueden sufrir

+ Overfitting: capturar patrones espurios que no se repetirán en el futuro, lo que conducirá a predicciones menos precisas, o
+ Underfitting: no capturar patrones relevantes, nuevamente conduce a predicciones menos precisas.

Utilizamos datos de **validación**, que no se utilizan en la capacitación de modelos, para medir la precisión de un modelo candidato. Esto nos permite probar muchos modelos candidatos y mantener el mejor.

## Ejercicio

Hemos creado nuestro primer modelo y ahora es el momento de optimizar el tamaño del árbol para hacer mejores predicciones.

In [2]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor


# Path del archivo a leer
iowa_file_path = "./input/melbourne-housing-snapshot/train.csv"

home_data = pd.read_csv(iowa_file_path)

# Crea Objetivo
y = home_data.SalePrice
# Crea Características
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

# Divide loas datos en validación y entrenamiento
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Especifica Modelo
iowa_model = DecisionTreeRegressor(random_state=1)
# Entrena Modelo
iowa_model.fit(train_X, train_y)

# Hace predicciones y calcula el error medio absoluto
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validación MAE: {:,.0f}".format(val_mae))

Validación MAE: 29,653


In [3]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae

### Paso 1: Comparar diferentes tamaños de árboles

Escribe un bucle que pruebe los siguientes valores para *max_leaf_nodes* a partir de un conjunto de valores posibles.

Llama a la función *get_mae* en cada valor de max_leaf_nodes. Almacena la salida de alguna manera que permita seleccionar el valor de `max_leaf_nodes` que proporcione el modelo más preciso en sus datos.

In [21]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
scores = []
for max_leaf_nodes in candidate_max_leaf_nodes:
    scores.append((get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y), max_leaf_nodes))
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(scores[-1][1], scores[-1][0]))

best_tree_size = min(scores)[1]
best_tree_size

Max leaf nodes: 5  		 Mean Absolute Error:  35044
Max leaf nodes: 25  		 Mean Absolute Error:  29016
Max leaf nodes: 50  		 Mean Absolute Error:  27405
Max leaf nodes: 100  		 Mean Absolute Error:  27282
Max leaf nodes: 250  		 Mean Absolute Error:  27893
Max leaf nodes: 500  		 Mean Absolute Error:  29454


100

### Paso 2: Entrenar el modelo usando todos los datos

Conoces el mejor tamaño de árbol. Si fueras a implementar este modelo en la práctica, lo harías aún más preciso utilizando todos los datos y manteniendo ese tamaño de árbol. Es decir, no necesitas retener los datos de validación ahora que has tomado todas tus decisiones de modelado.

In [22]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=0)

In [24]:
final_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=100, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

Has ajustado este modelo y has mejorado tus resultados. Pero todavía estamos usando los modelos de árbol de decisión, que no son muy sofisticados para los estándares modernos de aprendizaje automático. En el siguiente paso, aprenderemos a utilizar los bosques aleatorios para mejorar aún más nuestros modelos.